# Exploratory Data Analysis

In [1]:
# importing libraries
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

# adding scripts to path
sys.path.append(os.path.abspath("../scripts/python"))

In [2]:
# loading scripts
from manager import Manager
man = Manager()

In [3]:
# establishing connection

connection, cursor = man.connect_to_server(host="localhost", port=5432, user="warehouse", password="warehouse", dbName="warehouse")

successfully connected; cursor: <cursor object at 0x7fce08eebe50; closed: 0>


In [4]:
# fetching data
man.fetch_data(conn=connection, limit=10)

/home/bini/anaconda3/envs/py38/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,campaign_id,types,width,height,creative_id,auction_id,browser_ts,game_key,geo_country,site_name,...,delivery_requirements,cost_centre,currency,buy_rate_cpe,volume_agreed,gross_cost_or_budget,agency_fee,percentages,flat_fee,net_cost
0,sjliy81,impression,300,250,d2q570ur,869171bf-1711-485b-b907-be4fc28fb957,2021-05-12T20:02:00.911Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,www.vanityfair.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
1,sjliy81,impression,300,250,d2q570ur,48d13d36-b21f-4a09-85eb-43ea7d2a6d3e,2021-05-12T20:02:01.117Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,www.wlox.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
2,sjliy81,impression,300,250,d2q570ur,64dd93e0-0917-46e1-9505-9f08d29f2c2c,2021-05-12T20:02:01.308Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,thecinemaholic.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
3,sjliy81,impression,300,250,d2q570ur,f8fadc67-f5cc-4d53-8649-1729ac3206ee,2021-05-12T20:02:01.472Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,lakeshowlife.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
4,sjliy81,impression,300,250,d2q570ur,2416eab2-ab01-42ad-a5da-dd547b17b09e,2021-05-12T20:02:01.511Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,lakeshowlife.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
5,sjliy81,impression,300,250,d2q570ur,ae294390-c233-4357-b973-366cbf29fd77,2021-05-12T20:02:01.980Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,www.pioneersmokehouses.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
6,sjliy81,impression,300,250,d2q570ur,8ebb0a08-5480-43e2-9d3d-f224e5c6ff67,2021-05-12T20:02:02.765Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,www.pocketgamer.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
7,sjliy81,impression,300,250,d2q570ur,8b6b00cc-46f1-4287-b56d-525745762d42,2021-05-12T20:02:03.033Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,fortnitetracker.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
8,sjliy81,impression,300,250,d2q570ur,af5be49f-0200-4ed7-b902-99a42b3052d1,2021-05-12T20:02:03.166Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,boltbeat.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
9,sjliy81,impression,300,250,d2q570ur,4a08bd69-f2ff-4c19-8375-acdf19704f53,2021-05-12T20:02:03.516Z,adunit-lionsgate-spiral-new-2-puzzle-v3-mpu/6d...,United States,mix949.com,...,None,USA,USD,0.33,151515.15,50000,Percentage,0,None,50000
